In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from rapidfuzz import process, fuzz
from tqdm import tqdm
from pathlib import Path

In [2]:
# === Caminhos ===
DADOS_DIR = Path("Dados/")
RESULTADOS_DIR = Path("Resultados/")
RESULTADOS_DIR.mkdir(exist_ok=True)

# === Parâmetros ===
FUZZY_THRESHOLD = 90

# -------------------------------------------------------------
# 1️⃣ Funções auxiliares
# -------------------------------------------------------------

def limpar_titulo(titulo: str) -> str:
    """Normaliza o título: minúsculas, sem acento, sem partes de temporada."""
    if not isinstance(titulo, str):
        return ""
    t = titulo.lower().strip()
    t = unidecode(t)
    # remove partes de temporada e subtítulo
    t = re.sub(r":\s*(season|seasons|part|episode|ep|chapter|capitulo)\s*\d+", "", t)
    t = re.split(r":", t, maxsplit=2)[0].strip()
    t = re.sub(r"[\(\[].*?[\)\]]", "", t)
    t = re.sub(r"[^a-z0-9\s]", "", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

In [3]:
def fuzzy_match_catalogo(catalogo_df, imdb_df):
    """Aplica fuzzy matching entre catálogo Netflix e IMDB."""
    imdb_titulos = imdb_df["titulo_limpo"].tolist()
    matches = []

    for _, row in tqdm(catalogo_df.iterrows(), total=len(catalogo_df)):
        title = row["titulo_limpo"]
        match, score, _ = process.extractOne(title, imdb_titulos, scorer=fuzz.token_sort_ratio)
        if score >= FUZZY_THRESHOLD:
            matches.append((row["title"], match, score))
        else:
            matches.append((row["title"], None, score))

    return pd.DataFrame(matches, columns=["catalogo_title", "imdb_title", "match_score"])

In [4]:
# -------------------------------------------------------------
# 2️⃣ Função principal
# -------------------------------------------------------------

def unir_catalogo_imdb():
    print("🎞️ Carregando catálogo da Netflix e dados do IMDB...\n")

    # Catálogo Netflix (dataset do Kaggle)
    catalogo = pd.read_csv(DADOS_DIR / "netflix_titles.csv", encoding="utf-8")
    imdb = pd.read_parquet(RESULTADOS_DIR / "IMDB_Limpo.parquet", engine="fastparquet")

    print(f"📂 Catálogo: {len(catalogo):,} títulos")
    print(f"🎬 IMDB: {len(imdb):,} registros\n")

    # 1️⃣ Limpeza de títulos
    catalogo["titulo_limpo"] = catalogo["title"].apply(limpar_titulo)
    imdb["titulo_limpo"] = imdb["primaryTitle"].apply(limpar_titulo)

    # 2️⃣ Deduplicar IMDB para evitar múltiplos matches
    imdb_unico = (
        imdb.sort_values("numVotes", ascending=False)
            .drop_duplicates(subset=["titulo_limpo"], keep="first")
            .reset_index(drop=True)
    )

    # 3️⃣ Fuzzy matching
    print("🔍 Iniciando fuzzy matching entre catálogo e IMDB...")
    matches = fuzzy_match_catalogo(catalogo, imdb_unico)

    # 4️⃣ Merge final
    df_final = matches.merge(imdb_unico, left_on="imdb_title", right_on="titulo_limpo", how="left")

    # 5️⃣ Filtrar apenas notas válidas (averageRating ≠ -1)
    df_final = df_final[df_final["averageRating"] != -1].copy()

    # 6️⃣ Normalizar score (0–1)
    df_final["match_score"] = df_final["match_score"] / 100

    # 7️⃣ Salvar resultados
    parquet_path = RESULTADOS_DIR / "Catalogo_IMDB_Unido.parquet"
    csv_path = RESULTADOS_DIR / "Catalogo_IMDB_Unido.csv"

    df_final.to_parquet(parquet_path, index=False, engine="fastparquet")
    df_final.to_csv(csv_path, sep=";", decimal=",", index=False, encoding="utf-8-sig")

    print("\n✅ União concluída!")
    print(f"💾 Parquet salvo em: {parquet_path}")
    print(f"💾 CSV salvo em: {csv_path}")
    print(f"🎬 Total de títulos com nota válida: {len(df_final):,}")

    return df_final


# -------------------------------------------------------------
# 3️⃣ Execução
# -------------------------------------------------------------

df_catalogo_imdb = unir_catalogo_imdb()
df_catalogo_imdb.head(10)

🎞️ Carregando catálogo da Netflix e dados do IMDB...

📂 Catálogo: 8,807 títulos
🎬 IMDB: 1,170,931 registros

🔍 Iniciando fuzzy matching entre catálogo e IMDB...


100%|██████████| 8807/8807 [17:06<00:00,  8.58it/s]



✅ União concluída!
💾 Parquet salvo em: Resultados\Catalogo_IMDB_Unido.parquet
💾 CSV salvo em: Resultados\Catalogo_IMDB_Unido.csv
🎬 Total de títulos com nota válida: 8,673


,catalogo_title,imdb_title,match_score,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,titulo_limpo
0,Dick Johnson Is Dead,dick johnson is dead,1.0,tt11394180,movie,Dick Johnson Is Dead,Dick Johnson Is Dead,2020.0,89.0,"Biography,Documentary,Drama",7.4,7551.0,dick johnson is dead
1,Blood & Water,blood water,1.0,tt9839146,tvSeries,Blood & Water,Blood & Water,2020.0,53.0,"Drama,Mystery",6.7,4697.0,blood water
2,Ganglands,ganglands,1.0,tt13278100,tvSeries,Ganglands,Braqueurs,2021.0,44.0,"Action,Crime,Drama",7.2,4915.0,ganglands
3,Jailbirds New Orleans,jailbirds new orleans,1.0,tt15320436,tvSeries,Jailbirds New Orleans,Jailbirds New Orleans,2021.0,-1.0,"Documentary,Reality-TV",6.5,337.0,jailbirds new orleans
4,Kota Factory,kota factory,1.0,tt9432978,tvSeries,Kota Factory,Kota Factory,2019.0,40.0,"Comedy,Drama",9.0,89937.0,kota factory
5,Midnight Mass,midnight mass,1.0,tt0323246,movie,Midnight Mass,Midnight Mass,2003.0,98.0,Horror,2.6,494.0,midnight mass
6,My Little Pony: A New Generation,my little pony,1.0,tt1751105,tvSeries,My Little Pony: Friendship Is Magic,My Little Pony: Friendship Is Magic,2010.0,22.0,"Adventure,Animation,Comedy",7.8,25022.0,my little pony
7,Sankofa,sankofa,1.0,tt0108041,movie,Sankofa,Sankofa,1993.0,125.0,Drama,7.0,882.0,sankofa
8,The Great British Baking Show,the great british baking show,1.0,tt1877368,tvSeries,The Great British Baking Show,The Great British Bake Off,2010.0,60.0,"Game-Show,Reality-TV",8.6,14720.0,the great british baking show
9,The Starling,the starling,1.0,tt5164438,movie,The Starling,The Starling,2021.0,102.0,"Comedy,Drama",6.4,16576.0,the starling
